<a href="https://colab.research.google.com/github/kushalhole/Data_science_lab_SE_20/blob/main/ds2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Breadth-First Search (BFS)

BFS is an algorithm for traversing or searching tree or graph data structures. It starts at the tree root (or some arbitrary node of a graph, sometimes referred to as a 'search key') and explores all of the neighbor nodes at the present depth prior to moving on to the nodes at the next depth level.

It typically uses a queue data structure to keep track of the nodes to visit.

In [1]:
from collections import deque

def bfs(graph, start_node):
    visited = set()
    queue = deque([start_node])
    bfs_path = []

    visited.add(start_node)

    while queue:
        current_node = queue.popleft()
        bfs_path.append(current_node)

        for neighbor in graph[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    return bfs_path

# Example Graph (Adjacency List)
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print("BFS Traversal starting from node 'A':")
print(bfs(graph, 'A'))

BFS Traversal starting from node 'A':
['A', 'B', 'C', 'D', 'E', 'F']


### A* Search Algorithm

A* is a popular pathfinding algorithm often used in graph traversal and pathfinding. It is an informed search algorithm, meaning it uses a heuristic function to guide its search.

A* uses two main cost functions:
1.  **g(n)**: The cost from the start node to node `n`.
2.  **h(n)**: The estimated cost from node `n` to the goal node (the heuristic).

The algorithm then selects the path that minimizes **f(n) = g(n) + h(n)**.

It typically uses a priority queue to efficiently retrieve the node with the lowest `f(n)` value.

In [3]:
import heapq

def a_star_search(graph, start, goal, heuristic):
    # The set of discovered nodes that may need to be (re-)expanded.
    # Initially, only the start node is known.
    # This is a min-heap, so we use (f_score, node) to ensure priority queue behavior
    open_set = [(heuristic[start], start)]

    # For node n, came_from[n] is the node immediately preceding it on the cheapest path from start
    # currently known.
    came_from = {}

    # For node n, g_score[n] is the cost of the cheapest path from start to n currently known.
    g_score = {node: float('inf') for node in graph}
    g_score[start] = 0

    # For node n, f_score[n] = g_score[n] + heuristic[n]. f_score[n] represents our current best guess as to
    # how cheap a path from start to finish can be if it goes through n.
    f_score = {node: float('inf') for node in graph}
    f_score[start] = heuristic[start]

    while open_set:
        # Current node is the one with the lowest f_score
        current_f, current = heapq.heappop(open_set)

        if current == goal:
            # Reconstruct path if goal is reached
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        for neighbor, weight in graph[current].items():
            # d(current, neighbor) is the weight of the edge from current to neighbor
            tentative_g_score = g_score[current] + weight

            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic[neighbor]
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None # No path found

# Example Graph (Adjacency List with weights)
graph_astar = {
    'A': {'B': 1, 'C': 3},
    'B': {'A': 1, 'D': 3, 'E': 6},
    'C': {'A': 3, 'F': 2},
    'D': {'B': 3},
    'E': {'B': 6, 'F': 1},
    'F': {'C': 2, 'E': 1}
}

# Heuristic values (estimated cost from node to goal 'F')
# For simplicity, these are made up for demonstration purposes
heuristic = {
    'A': 7, # A to F
    'B': 6, # B to F
    'C': 2, # C to F
    'D': 8, # D to F
    'E': 1, # E to F
    'F': 0  # F to F
}

start_node_astar = 'A'
goal_node_astar = 'F'

print(f"A* Search Traversal from {start_node_astar} to {goal_node_astar}:")
path = a_star_search(graph_astar, start_node_astar, goal_node_astar, heuristic)

if path:
    print("Path found:", path)
else:
    print("No path found.")

A* Search Traversal from A to F:
Path found: ['A', 'C', 'F']


### Minimax Algorithm

The Minimax algorithm is a decision-making algorithm used in game theory for two-player, zero-sum games (meaning one player's gain is another's loss). It's typically used in games like Tic-Tac-Toe, chess, and checkers.

The algorithm works by recursively exploring all possible game states from the current state to a certain depth (or until a terminal state is reached). It assumes that both players play optimally: the 'maximizing' player tries to maximize their score, while the 'minimizing' player tries to minimize the maximizing player's score.

**Key components:**
*   **Game State:** A representation of the current configuration of the game.
*   **Terminal Test:** A function to check if a game state is over.
*   **Utility Function:** A function that assigns a numerical value to a terminal state, representing the outcome for the maximizing player.
*   **Successor Function:** A function that returns all possible next states from a given state, along with the moves that lead to them.

We will demonstrate it with a simple Tic-Tac-Toe game.

In [4]:
import math

# --- Tic-Tac-Toe Game Definition ---

# Represents the board as a 1D list, 0-8
# ' ' for empty, 'X' for player X, 'O' for player O
def create_board():
    return [' '] * 9

def display_board(board):
    print(f"{board[0]}|{board[1]}|{board[2]}")
    print(f"-+-+-")
    print(f"{board[3]}|{board[4]}|{board[5]}")
    print(f"-+-+-")
    print(f"{board[6]}|{board[7]}|{board[8]}")

def get_empty_cells(board):
    return [i for i, cell in enumerate(board) if cell == ' ']

def apply_move(board, move, player):
    new_board = list(board) # Create a copy to avoid modifying the original
    new_board[move] = player
    return new_board

def check_win(board, player):
    win_conditions = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8), # Rows
        (0, 3, 6), (1, 4, 7), (2, 5, 8), # Columns
        (0, 4, 8), (2, 4, 6)             # Diagonals
    ]
    for condition in win_conditions:
        if all(board[i] == player for i in condition):
            return True
    return False

def is_terminal(board):
    return check_win(board, 'X') or check_win(board, 'O') or ' ' not in board

def utility(board):
    if check_win(board, 'X'):
        return 1  # X (Maximizing player) wins
    elif check_win(board, 'O'):
        return -1 # O (Minimizing player) wins
    else:
        return 0  # Draw

# --- Minimax Algorithm Implementation ---

def minimax(board, depth, maximizing_player):
    if is_terminal(board):
        return utility(board)

    if maximizing_player:
        max_eval = -math.inf
        for move in get_empty_cells(board):
            new_board = apply_move(board, move, 'X')
            eval = minimax(new_board, depth + 1, False)
            max_eval = max(max_eval, eval)
        return max_eval
    else:
        min_eval = math.inf
        for move in get_empty_cells(board):
            new_board = apply_move(board, move, 'O')
            eval = minimax(new_board, depth + 1, True)
            min_eval = min(min_eval, eval)
        return min_eval

def find_best_move(board):
    best_eval = -math.inf
    best_move = -1
    for move in get_empty_cells(board):
        new_board = apply_move(board, move, 'X')
        # Minimax assumes the opponent (False) will play optimally after our move
        eval = minimax(new_board, 0, False)
        if eval > best_eval:
            best_eval = eval
            best_move = move
    return best_move

# --- Example Usage (Tic-Tac-Toe) ---

print("\n--- Minimax for Tic-Tac-Toe ---")
current_board = create_board()
display_board(current_board)

# Let's simulate a few moves and see Minimax in action
# Player X (Minimax AI) starts

print("Minimax (X) makes a move:")
best_move = find_best_move(current_board)
current_board = apply_move(current_board, best_move, 'X')
display_board(current_board)

print("\nPlayer O (human) moves to 4:")
current_board = apply_move(current_board, 4, 'O')
display_board(current_board)

print("\nMinimax (X) makes a move:")
best_move = find_best_move(current_board)
current_board = apply_move(current_board, best_move, 'X')
display_board(current_board)

print("\nPlayer O (human) moves to 2:")
current_board = apply_move(current_board, 2, 'O')
display_board(current_board)

print("\nMinimax (X) makes a move:")
best_move = find_best_move(current_board)
current_board = apply_move(current_board, best_move, 'X')
display_board(current_board)

print(f"\nGame Over. Utility: {utility(current_board)}")


--- Minimax for Tic-Tac-Toe ---
 | | 
-+-+-
 | | 
-+-+-
 | | 
Minimax (X) makes a move:
X| | 
-+-+-
 | | 
-+-+-
 | | 

Player O (human) moves to 4:
X| | 
-+-+-
 |O| 
-+-+-
 | | 

Minimax (X) makes a move:
X|X| 
-+-+-
 |O| 
-+-+-
 | | 

Player O (human) moves to 2:
X|X|O
-+-+-
 |O| 
-+-+-
 | | 

Minimax (X) makes a move:
X|X|O
-+-+-
 |O| 
-+-+-
X| | 

Game Over. Utility: 0


### Depth-First Search (DFS)

DFS is another algorithm for traversing or searching tree or graph data structures. The algorithm starts at the root node (selecting some arbitrary node as the root node in the case of a graph) and explores as far as possible along each branch before backtracking.

It typically uses a stack data structure (implicitly through recursion or explicitly) to keep track of the nodes to visit.

In [2]:
def dfs(graph, start_node):
    visited = set()
    dfs_path = []

    def dfs_recursive(node):
        visited.add(node)
        dfs_path.append(node)

        for neighbor in graph[node]:
            if neighbor not in visited:
                dfs_recursive(neighbor)

    dfs_recursive(start_node)
    return dfs_path

# Using the same example graph
# graph = {
#     'A': ['B', 'C'],
#     'B': ['A', 'D', 'E'],
#     'C': ['A', 'F'],
#     'D': ['B'],
#     'E': ['B', 'F'],
#     'F': ['C', 'E']
# }

print("\nDFS Traversal starting from node 'A':")
print(dfs(graph, 'A'))


DFS Traversal starting from node 'A':
['A', 'B', 'D', 'E', 'F', 'C']
